In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

In [2]:
gdf = gpd.read_file(f"zip://statistical-gis-boundaries-london.zip!statistical-gis-boundaries-london/ESRI/MSOA_2011_London_gen_MHW.shp")
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   MSOA11CD   983 non-null    object  
 1   MSOA11NM   983 non-null    object  
 2   LAD11CD    983 non-null    object  
 3   LAD11NM    983 non-null    object  
 4   RGN11CD    983 non-null    object  
 5   RGN11NM    983 non-null    object  
 6   USUALRES   983 non-null    int64   
 7   HHOLDRES   983 non-null    int64   
 8   COMESTRES  983 non-null    int64   
 9   POPDEN     983 non-null    float64 
 10  HHOLDS     983 non-null    int64   
 11  AVHHOLDSZ  983 non-null    float64 
 12  geometry   983 non-null    geometry
dtypes: float64(2), geometry(1), int64(4), object(6)
memory usage: 100.0+ KB


In [3]:
gdf.to_crs("EPSG:4326", inplace = True)

In [7]:
#gdf.plot(figsize = (10,15), facecolor = "None", edgecolor = "r")

In [4]:
gdf

,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,geometry
0,E02000001,City of London 001,E09000001,City of London,E12000007,London,7375,7187,188,25.5,4385,1.6,"MULTIPOLYGON (((-0.10414 51.50841, -0.10444 51..."
1,E02000002,Barking and Dagenham 001,E09000002,Barking and Dagenham,E12000007,London,6775,6724,51,31.3,2713,2.5,"POLYGON ((0.14809 51.59678, 0.14806 51.59640, ..."
2,E02000003,Barking and Dagenham 002,E09000002,Barking and Dagenham,E12000007,London,10045,10033,12,46.9,3834,2.6,"POLYGON ((0.15063 51.58306, 0.14838 51.58075, ..."
3,E02000004,Barking and Dagenham 003,E09000002,Barking and Dagenham,E12000007,London,6182,5937,245,24.8,2318,2.6,"POLYGON ((0.18508 51.56480, 0.18400 51.56391, ..."
4,E02000005,Barking and Dagenham 004,E09000002,Barking and Dagenham,E12000007,London,8562,8562,0,72.1,3183,2.7,"POLYGON ((0.14988 51.56807, 0.15076 51.56778, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,E02006927,Greenwich 034,E09000011,Greenwich,E12000007,London,8315,8241,74,33.0,3338,2.5,"POLYGON ((0.02897 51.46779, 0.02993 51.46592, ..."
979,E02006928,Greenwich 035,E09000011,Greenwich,E12000007,London,7341,6410,931,136.0,2977,2.2,"MULTIPOLYGON (((-0.00964 51.48366, -0.00981 51..."
980,E02006929,Greenwich 036,E09000011,Greenwich,E12000007,London,7490,7489,1,29.4,3333,2.2,"POLYGON ((0.01616 51.49578, 0.01852 51.49498, ..."
981,E02006930,Greenwich 037,E09000011,Greenwich,E12000007,London,6561,6557,4,75.6,2876,2.3,"POLYGON ((0.00864 51.48917, 0.00834 51.48877, ..."


In [8]:
gdf["Centroids"] = gdf.geometry.centroid

/opt/conda/envs/sds2020/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [9]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
datasave2 = pd.DataFrame(gdf)
datasave2.to_csv('map.csv')

In [10]:
gdf.set_index('MSOA11CD', inplace=True, drop=True)

distances = gdf.rename_axis('Orig').Centroids.apply(lambda x: gdf.rename_axis('Dest').Centroids.distance(x)).stack().reset_index()
distances.rename(columns = {0:'distance'}, inplace=True)
distances.sort_values(by=["Orig", "Dest"], inplace = True)
gdf.reset_index(inplace=True)

/opt/conda/envs/sds2020/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
distances

,Orig,Dest,distance
0,E02000001,E02000001,0.000000
1,E02000001,E02000002,0.242971
2,E02000001,E02000003,0.240652
3,E02000001,E02000004,0.272012
4,E02000001,E02000005,0.240019
...,...,...,...
966284,E02006931,E02006927,0.034254
966285,E02006931,E02006928,0.013793
966286,E02006931,E02006929,0.024047
966287,E02006931,E02006930,0.019682


In [11]:
datasave = pd.DataFrame(distances)
datasave.to_csv('distancce matrix.csv')

In [28]:
temp = 0
centerpoint = []

In [31]:
while temp<=982:
    centerpoint.append(gdf["Centroids"][temp])
    temp = temp+1


In [12]:
centerpoint=gdf["Centroids"]

In [13]:
type(centerpoint)

geopandas.geoseries.GeoSeries

In [14]:
type(centerpoint[0])

shapely.geometry.point.Point

In [15]:
gdf

,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,geometry,Centroids
0,E02000001,City of London 001,E09000001,City of London,E12000007,London,7375,7187,188,25.5,4385,1.6,"MULTIPOLYGON (((-0.10414 51.50841, -0.10444 51...",POINT (-0.09213 51.51482)
1,E02000002,Barking and Dagenham 001,E09000002,Barking and Dagenham,E12000007,London,6775,6724,51,31.3,2713,2.5,"POLYGON ((0.14809 51.59678, 0.14806 51.59640, ...",POINT (0.13948 51.58827)
2,E02000003,Barking and Dagenham 002,E09000002,Barking and Dagenham,E12000007,London,10045,10033,12,46.9,3834,2.6,"POLYGON ((0.15063 51.58306, 0.14838 51.58075, ...",POINT (0.14090 51.57493)
3,E02000004,Barking and Dagenham 003,E09000002,Barking and Dagenham,E12000007,London,6182,5937,245,24.8,2318,2.6,"POLYGON ((0.18508 51.56480, 0.18400 51.56391, ...",POINT (0.17683 51.55548)
4,E02000005,Barking and Dagenham 004,E09000002,Barking and Dagenham,E12000007,London,8562,8562,0,72.1,3183,2.7,"POLYGON ((0.14988 51.56807, 0.15076 51.56778, ...",POINT (0.14332 51.56142)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,E02006927,Greenwich 034,E09000011,Greenwich,E12000007,London,8315,8241,74,33.0,3338,2.5,"POLYGON ((0.02897 51.46779, 0.02993 51.46592, ...",POINT (0.02366 51.45811)
979,E02006928,Greenwich 035,E09000011,Greenwich,E12000007,London,7341,6410,931,136.0,2977,2.2,"MULTIPOLYGON (((-0.00964 51.48366, -0.00981 51...",POINT (-0.01841 51.48075)
980,E02006929,Greenwich 036,E09000011,Greenwich,E12000007,London,7490,7489,1,29.4,3333,2.2,"POLYGON ((0.01616 51.49578, 0.01852 51.49498, ...",POINT (0.01164 51.49339)
981,E02006930,Greenwich 037,E09000011,Greenwich,E12000007,London,6561,6557,4,75.6,2876,2.3,"POLYGON ((0.00864 51.48917, 0.00834 51.48877, ...",POINT (0.01172 51.48618)


---

In [13]:
for index, row in distances:
    if row['distance'] == 0:
        distances.drop(labels=distances.iloc[index,0] ,axis=0)

ValueError: not enough values to unpack (expected 2, got 1)

In [16]:
data0 = pd.read_csv('result1.csv')
data0

,Unnamed: 0,index,ECODE,BUNGALOW,FLAT_MAIS,HOUSE_TERRACED,ALL_PROPERTIES,id,price,traffic,crime rate,ptai,group
0,0,0,E02000001,5.0,6540.0,90.0,6680.0,E02000001,203,168,0.034836,11.767474,4
1,1,1,E02000002,310.0,1100.0,1060.0,2770.0,E02000002,60,20,0.011596,3.675595,3
2,2,2,E02000003,110.0,1280.0,1880.0,3970.0,E02000003,137,18,0.009579,3.675595,1
3,3,3,E02000004,110.0,310.0,1260.0,2380.0,E02000004,77,13,0.014535,3.600575,3
4,4,4,E02000005,40.0,390.0,2800.0,3290.0,E02000005,128,10,0.008547,8.159091,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,979,979,E02000980,0.0,4880.0,120.0,5070.0,E02000980,207,50,0.008307,66.445087,4
980,980,980,E02000981,0.0,4420.0,470.0,5020.0,E02000981,139,44,0.008235,42.144928,1
981,981,981,E02000982,0.0,4460.0,430.0,5100.0,E02000982,88,49,0.007472,64.736842,7
982,982,982,E02000983,0.0,4630.0,120.0,4870.0,E02000983,69,41,0.008910,26.758065,3


In [17]:
result2= pd.merge(data0, gdf ,left_on='ECODE', right_on='MSOA11CD', how='left').reset_index()

In [18]:
result2

,level_0,Unnamed: 0,index,ECODE,BUNGALOW,FLAT_MAIS,HOUSE_TERRACED,ALL_PROPERTIES,id,price,...,RGN11CD,RGN11NM,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,geometry,Centroids
0,0,0,0,E02000001,5.0,6540.0,90.0,6680.0,E02000001,203,...,E12000007,London,7375.0,7187.0,188.0,25.5,4385.0,1.6,"MULTIPOLYGON (((-0.10414 51.50841, -0.10444 51...",POINT (-0.09213 51.51482)
1,1,1,1,E02000002,310.0,1100.0,1060.0,2770.0,E02000002,60,...,E12000007,London,6775.0,6724.0,51.0,31.3,2713.0,2.5,"POLYGON ((0.14809 51.59678, 0.14806 51.59640, ...",POINT (0.13948 51.58827)
2,2,2,2,E02000003,110.0,1280.0,1880.0,3970.0,E02000003,137,...,E12000007,London,10045.0,10033.0,12.0,46.9,3834.0,2.6,"POLYGON ((0.15063 51.58306, 0.14838 51.58075, ...",POINT (0.14090 51.57493)
3,3,3,3,E02000004,110.0,310.0,1260.0,2380.0,E02000004,77,...,E12000007,London,6182.0,5937.0,245.0,24.8,2318.0,2.6,"POLYGON ((0.18508 51.56480, 0.18400 51.56391, ...",POINT (0.17683 51.55548)
4,4,4,4,E02000005,40.0,390.0,2800.0,3290.0,E02000005,128,...,E12000007,London,8562.0,8562.0,0.0,72.1,3183.0,2.7,"POLYGON ((0.14988 51.56807, 0.15076 51.56778, ...",POINT (0.14332 51.56142)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,979,979,979,E02000980,0.0,4880.0,120.0,5070.0,E02000980,207,...,E12000007,London,8684.0,8626.0,58.0,193.7,4498.0,1.9,"POLYGON ((-0.12691 51.49485, -0.12468 51.49471...",POINT (-0.13097 51.49209)
980,980,980,980,E02000981,0.0,4420.0,470.0,5020.0,E02000981,139,...,E12000007,London,8991.0,8866.0,125.0,247.2,4853.0,1.8,"POLYGON ((-0.13893 51.49254, -0.13759 51.49109...",POINT (-0.14154 51.49061)
981,981,981,981,E02000982,0.0,4460.0,430.0,5100.0,E02000982,88,...,E12000007,London,8226.0,8198.0,28.0,126.0,4143.0,2.0,"POLYGON ((-0.14343 51.49297, -0.14462 51.49221...",POINT (-0.14822 51.49076)
982,982,982,982,E02000983,0.0,4630.0,120.0,4870.0,E02000983,69,...,E12000007,London,8434.0,8389.0,45.0,223.4,4422.0,1.9,"MULTIPOLYGON (((-0.14551 51.48564, -0.14899 51...",POINT (-0.13613 51.48730)


In [20]:
datasave2 = pd.DataFrame(result2)
datasave2.to_csv('result3.csv')